### Items based CF 

### Import libraries 

In [101]:
import pandas as pd
from itertools import chain, combinations
import numpy as np
import matplotlib.pyplot as plt
from surprise import Reader, Dataset, SVD, BaselineOnly, NMF, accuracy
from surprise.model_selection import train_test_split
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error
from math import sqrt
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)
# pd.set_option('display.width', 1000)

### Import the ratings data file 

In [52]:
# read the ratings file 
# ratings = pd.read_csv('../data/ratings.csv', low_memory=False).set_index('CUSTOMER_ID')
ratings = pd.read_csv('../data/ratings.csv', index_col=0)
ratings.shape
# ratings.head(n=3)

(100282, 14)

In [53]:
# taking only a fraction of the data for computation resources
# take almost all the data for item and very few rows for users
ratings_sample = ratings.sample(frac=0.5)
ratings_sample.shape

(50141, 14)

In [ ]:
### Splitting the data to test and train sets

In [54]:
# splitting the sample data set  for evaluation later
# train_data, test_data = train_test_split(ratings_sample, test_size=0.25)
# train_data.shape, test_data.shape
# splitting the whole data set  for evaluation later
train_data, test_data = train_test_split(ratings, test_size=0.25)
train_data.shape, test_data.shape

((75211, 14), (25071, 14))

In [55]:
# computing the similarity matrix for items 
item_similarity_cos = cosine_similarity(ratings.T)
item_similarity_cos

array([[1.        , 0.05237787, 0.04820894, 0.08641488, 0.05472308,
        0.0530396 , 0.06202908, 0.06183807, 0.05294445, 0.0700065 ,
        0.05343616, 0.0550799 , 0.14309924, 0.14203105],
       [0.05237787, 1.        , 0.03343972, 0.04547429, 0.03550416,
        0.03225516, 0.21390307, 0.17499101, 0.03884455, 0.12242787,
        0.04048048, 0.04109852, 0.05953457, 0.05361255],
       [0.04820894, 0.03343972, 1.        , 0.0412513 , 0.03700454,
        0.03497993, 0.04155604, 0.03956295, 0.1260945 , 0.04144512,
        0.031816  , 0.03165361, 0.05890403, 0.0397234 ],
       [0.08641488, 0.04547429, 0.0412513 , 1.        , 0.15158669,
        0.13424858, 0.05500565, 0.04841213, 0.04473979, 0.05689133,
        0.14439348, 0.16214569, 0.18448123, 0.15892044],
       [0.05472308, 0.03550416, 0.03700454, 0.15158669, 1.        ,
        0.14197596, 0.04173009, 0.0369634 , 0.03692043, 0.04364928,
        0.12554604, 0.18434204, 0.38980126, 0.13832433],
       [0.0530396 , 0.03225516, 0.0

In [56]:
# create a dataframe of the correlation matrix for the orders
item_similarity_cos_df = pd.DataFrame(item_similarity_cos, index=ratings.columns,
                                 columns=ratings.columns)
item_similarity_cos_df

,YAKER_TREATS_ITEMS,SUPER_DUCK_TREATS_ITEMS,WHIMZEES_BONE_TREATS_ITEMS,CHICKEN_TREATS_ITEMS,DUCK_TREATS_ITEMS,SALMON_TREATS_ITEMS,SUPER_LAMB_TREATS_ITEMS,SUPER_SALMON_TREATS_ITEMS,WHIMZEES_TREATS_ITEMS,COLD_PRESSED_DUCK_TREATS_ITEMS,PORK_TREATS_ITEMS,LAMB_TREATS_ITEMS,DENTAL_DAILIES_ITEMS,GOOD_DOG_TREATS_ITEMS
YAKER_TREATS_ITEMS,1.000000,0.052378,0.048209,0.086415,0.054723,0.053040,0.062029,0.061838,0.052944,0.070007,0.053436,0.055080,0.143099,0.142031
SUPER_DUCK_TREATS_ITEMS,0.052378,1.000000,0.033440,0.045474,0.035504,0.032255,0.213903,0.174991,0.038845,0.122428,0.040480,0.041099,0.059535,0.053613
WHIMZEES_BONE_TREATS_ITEMS,0.048209,0.033440,1.000000,0.041251,0.037005,0.034980,0.041556,0.039563,0.126095,0.041445,0.031816,0.031654,0.058904,0.039723
CHICKEN_TREATS_ITEMS,0.086415,0.045474,0.041251,1.000000,0.151587,0.134249,0.055006,0.048412,0.044740,0.056891,0.144393,0.162146,0.184481,0.158920
DUCK_TREATS_ITEMS,0.054723,0.035504,0.037005,0.151587,1.000000,0.141976,0.041730,0.036963,0.036920,0.043649,0.125546,0.184342,0.389801,0.138324
SALMON_TREATS_ITEMS,0.053040,0.032255,0.034980,0.134249,0.141976,1.000000,0.034910,0.048437,0.035810,0.041167,0.106532,0.136270,0.118583,0.072899
SUPER_LAMB_TREATS_ITEMS,0.062029,0.213903,0.041556,0.055006,0.041730,0.034910,1.000000,0.182852,0.041483,0.182921,0.047248,0.052545,0.068085,0.067507
SUPER_SALMON_TREATS_ITEMS,0.061838,0.174991,0.039563,0.048412,0.036963,0.048437,0.182852,1.000000,0.044187,0.164341,0.040278,0.035470,0.065966,0.058417
WHIMZEES_TREATS_ITEMS,0.052944,0.038845,0.126095,0.044740,0.036920,0.035810,0.041483,0.044187,1.000000,0.053099,0.035270,0.035975,0.046794,0.037159
COLD_PRESSED_DUCK_TREATS_ITEMS,0.070007,0.122428,0.041445,0.056891,0.043649,0.041167,0.182921,0.164341,0.053099,1.000000,0.045148,0.046079,0.069832,0.063205


In [8]:
# visualize the cosine matrix
# labels = []
# for item in item_similarity_cos_df:
#     labels.append(item)

# fig, ax = plt.subplots(figsize=(20,20))
# cax = ax.matshow(item_similarity_cos, interpolation='nearest')
# ax.grid(True)
# plt.title('Item Similarity matrix - Cosine')
# plt.xticks(range(14), labels, rotation=90);
# plt.yticks(range(14), labels);
# fig.colorbar(cax, ticks=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, .75,.8,.85,.90,.95,1])
# # plt.savefig('similarity_matrix.png',bbox_inches='tight', dpi=100)
# # plt.show()

In [57]:
# creating a similarity score for one user using cosine
def get_similar_prod_cos(item_name, customer_rating):
    similar_prod = item_similarity_cos_df[item_name]*(customer_rating)
    similar_prod = similar_prod.sort_values(ascending=False)
    return similar_prod


In [9]:
train_data.head(n=2)

,YAKER_TREATS_ITEMS,SUPER_DUCK_TREATS_ITEMS,WHIMZEES_BONE_TREATS_ITEMS,CHICKEN_TREATS_ITEMS,DUCK_TREATS_ITEMS,SALMON_TREATS_ITEMS,SUPER_LAMB_TREATS_ITEMS,SUPER_SALMON_TREATS_ITEMS,WHIMZEES_TREATS_ITEMS,COLD_PRESSED_DUCK_TREATS_ITEMS,PORK_TREATS_ITEMS,LAMB_TREATS_ITEMS,DENTAL_DAILIES_ITEMS,GOOD_DOG_TREATS_ITEMS
CUSTOMER_ID,,,,,,,,,,,,,,
4308866,0,0,0,0,0,3,0,0,0,0,0,0,3,0
3777603,1,0,0,0,1,0,0,0,0,0,0,0,4,2


In [59]:
# for customer x 
# customer = [('YAKER_TREATS_ITEMS',5),('SUPER_DUCK_TREATS_ITEMS',2)]
# random_id = np.random.choice(train_data.shape[0])
customer = ratings.filter(items=[3127],axis=0)
similar_prod = pd.DataFrame()
for prod,rating in customer.iteritems():
    similar_prod = similar_prod.append(get_similar_prod_cos(prod,rating.values),
                                       ignore_index=True)
similar_prod.sum().sort_values(ascending=False)

DENTAL_DAILIES_ITEMS              5.766767
GOOD_DOG_TREATS_ITEMS             5.047600
CHICKEN_TREATS_ITEMS              3.435350
DUCK_TREATS_ITEMS                 2.474050
YAKER_TREATS_ITEMS                2.224359
SALMON_TREATS_ITEMS               2.014566
LAMB_TREATS_ITEMS                 1.673033
PORK_TREATS_ITEMS                 1.045272
COLD_PRESSED_DUCK_TREATS_ITEMS    0.693900
SUPER_LAMB_TREATS_ITEMS           0.681808
SUPER_SALMON_TREATS_ITEMS         0.646216
SUPER_DUCK_TREATS_ITEMS           0.574558
WHIMZEES_BONE_TREATS_ITEMS        0.520478
WHIMZEES_TREATS_ITEMS             0.476884
dtype: float64

In [112]:
# for customer x 
# customer_std = orders_std.filter(items=[3127],axis=0)
customer = ratings.filter(items=[3127],axis=0)
similar_item = pd.DataFrame()
for prod,rating in customer.iteritems():
    similar_item = similar_item.append(get_similar_prod_cos(prod,rating.values),
                                        ignore_index=True)
item=[]    
for i,v in customer.iteritems():
    if v.values==0:
        item.append(i)          
    
similar_item=similar_item[item]   
print("The recommended items are: \n",format(similar_item.sum().sort_values(ascending=True)))

The recommended items are: 
 WHIMZEES_TREATS_ITEMS             0.476884
WHIMZEES_BONE_TREATS_ITEMS        0.520478
SUPER_DUCK_TREATS_ITEMS           0.574558
SUPER_SALMON_TREATS_ITEMS         0.646216
SUPER_LAMB_TREATS_ITEMS           0.681808
COLD_PRESSED_DUCK_TREATS_ITEMS    0.693900
PORK_TREATS_ITEMS                 1.045272
LAMB_TREATS_ITEMS                 1.673033
DUCK_TREATS_ITEMS                 2.474050
dtype: float64


### Evaluating the ratings based model using RMSE

In [93]:
n_users = ratings.shape[0]
n_items = ratings.shape[1]
print('Number of customers : {}\tNumber of products : {}'.format(n_users, n_items))

Number of customers : 100282	Number of products : 14


In [95]:
train_data.shape

(75211, 14)

In [94]:
# Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]


In [97]:
# Using cosine similarity
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [98]:
prediction = train_data_matrix.dot(item_similarity) / np.array([np.abs(item_similarity).sum(axis=1)])


In [104]:
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))


In [106]:
errors = rmse(prediction, test_data_matrix)

In [107]:
print('Item-based CF RMSE: {}'.format(errors))
print("Accuracy: {:.3f}".format(100-errors))

Item-based CF RMSE: 1.9230769230769231
Accuracy: 98.077
